<a href="https://colab.research.google.com/github/Jimmy-Rais/Machine-Learning/blob/main/SmartGridStability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
import tensorflow as tf

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving smart_grid_stability_augmented.csv to smart_grid_stability_augmented.csv


In [ ]:
import pandas as pd
import io
df=pd.read_csv(io.BytesIO(uploaded['smart_grid_stability_augmented.csv']))

In [153]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [158]:
#Preprocessing
df=df.dropna()
x=df.drop(['stabf', 'stab'], axis=1)
y=df['stabf']
data_scaled=preprocessing.MinMaxScaler(feature_range=(0,1))
x_scaled=data_scaled.fit_transform(x)


In [183]:
#Modeling
#1.Logistic regression hyperparameters tuning using GridSearchCv
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
x_train,x_t,y_train,y_t=train_test_split(x,y,test_size=0.3,random_state=0)
#Validation set(cv) and Test set
x_cv,x_test,y_cv,y_test=train_test_split(x_t,y_t,test_size=0.2,random_state=0)
print(f"Training set size{x_train.shape}")
print(f"Validation set size{x_cv.shape}")
print(f"Test set size{x_test.shape}")
model=LogisticRegression(multi_class='auto',max_iter=1000)
params={
    'C':[0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    'solver':['lbfgs','saga','sag','newton-cholesky'],
    'penalty':['l1','l2','elasticnet']
}
LogisticR_cv=GridSearchCV(model,params,cv=5,scoring='accuracy')
LogisticR_cv.fit(x_train,y_train)
print(LogisticR_cv.best_score_)
print(LogisticR_cv.best_params_)

Training set size(42000, 12)
Validation set size(14400, 12)
Test set size(3600, 12)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
385 fits failed out of a total of 660.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Sol

0.8152142857142856
{'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}


In [184]:
#Validation test
print(f"Validation_socre with Logistic regression:{LogisticR_cv.score(x_cv,y_cv)}")
#Test set
print(f"Test_score with Logistic Regression:{LogisticR_cv.score(x_test,y_test)}")


Validation_socre with Logistic regression:0.8174305555555555
Test_score with Logistic Regression:0.8008333333333333


In [185]:
#Xgboost
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#Encoding labels
y_train_2=le.fit_transform(y_train)
y_cv_2=le.fit_transform(y_cv)
y_test_2=le.fit_transform(y_test)
import xgboost as xgb
clf=xgb.XGBClassifier()
clf.fit(x_train,y_train_2)
clf.score(x_cv,y_cv_2)
print(f"Validation_socre with XGBoost:{clf.score(x_cv,y_cv_2)}")
#Test set
print(f"Test_score with XGBoost:{clf.score(x_test,y_test_2)}")

Validation_socre with XGBoost:0.9770138888888888
Test_score with XGBoost:0.9780555555555556
